## ResNet

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import os
import time 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns


def initialization():
    keras.backend.clear_session()
    np.random.seed(42)
    tf.random.set_seed(42)

### 使用Keras实现ResNet CNN

In [5]:
initialization()

In [6]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Softmax, BatchNormalization, ReLU, GlobalAvgPool2D, Add, Dropout
from tensorflow.keras.models import Model, Sequential

#### 实现残差结构

In [7]:
# 定义残差模块-基本形式
class Residual_Basic(keras.layers.Layer):
    Expansion = 1  # 扩展系数  -> 默认不使用虚线结构

    def __init__(self, filters, strides=1, down_sample=None, **kwargs):
        super(Residual_Basic, self).__init__(**kwargs)
        self.conv1 = Conv2D(filters=filters,
                            kernel_size=3,
                            strides=strides,
                            padding="SAME",
                            use_bias=False)
        self.BN1 = BatchNormalization(momentum=0.9, epsilon=1e-5)

        self.conv2 = Conv2D(filters=filters,
                            kernel_size=3,
                            strides=1,
                            padding="SAME",
                            use_bias=False)
        self.BN2 = BatchNormalization(momentum=0.9, epsilon=1e-5)
        # down_sample：使用改变特征图大小核深度的跳过连接
        self.down_sample = down_sample
        self.relu = ReLU()
        self.add = Add()

    def call(self, inputs, training=False):
        # 跳过连接分支
        skip_Z = inputs

        # 使用改变特征图大小核深度的跳过连接分支
        if self.down_sample is not None:
            skip_Z = self.down_sample(inputs)

        # 主分支
        Z = self.conv1(inputs)
        Z = self.BN1(Z, training=training)
        Z = self.relu(Z)

        Z = self.conv2(Z)
        Z = self.BN2(Z, training=training)

        Z = self.add([Z, skip_Z])
        Z = self.relu(Z)

        return Z

In [8]:
# 定义残差模块-瓶颈层形式
class Residual_Bottleneck(keras.layers.Layer):
    Expansion = 4  # 扩展系数

    def __init__(self, filters, strides=1, down_sample=None, **kwargs):
        super(Residual_Bottleneck, self).__init__(**kwargs)
        # 1×1卷积核:降低特征维度
        self.conv1 = Conv2D(filters=filters,
                            kernel_size=1,
                            use_bias=False,
                            name="conv1")  # 名字用于在迁移学习中与预训练模型的层进行匹配
        self.BN1 = BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv1/BatchNorm")

        self.conv2 = Conv2D(filters=filters,
                            kernel_size=3,
                            strides=strides,
                            padding="SAME",
                            use_bias=False,
                            name="conv2")
        self.BN2 = BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv2/BatchNorm")

        # 1×1卷积核:升高特征维度
        self.conv3 = Conv2D(filters=filters * self.Expansion,  # 64->256  128->512  ...
                            kernel_size=1,
                            use_bias=False,
                            name="conv3")
        self.BN3 = BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv3/BatchNorm")
        # down_sample：使用改变特征图大小核深度的跳过连接
        self.down_sample = down_sample
        self.relu = ReLU()
        self.add = Add()

    def call(self, inputs, training=False):
        # 跳过连接分支
        skip_Z = inputs

        # 使用改变特征图大小核深度的跳过连接分支
        if self.down_sample is not None:
            skip_Z = self.down_sample(inputs)

        # 主分支
        Z = self.conv1(inputs)
        Z = self.BN1(Z, training=training)
        Z = self.relu(Z)

        Z = self.conv2(Z)
        Z = self.BN2(Z, training=training)
        Z = self.relu(Z)

        Z = self.conv3(Z)
        Z = self.BN3(Z, training=training)

        Z = self.add([Z, skip_Z])
        Z = self.relu(Z)

        return Z

#### 生成一系列的残差结构

In [9]:
def make_conv_x(block, block_num, in_channel, unit1_channel, name, strides=1):
    """
    :param block: 可选择 Residual_Basic 或 Residual_Bottleneck
    :param block_num: 残差结构数量
    :param in_channel: 上一层输出特征矩阵的通道数
    :param unit1_channel: 本残差模块第一个单元的卷积层的的通道数
    """
    # 使用改变特征图大小核深度的跳过连接分支(虚线结构)
    # 1. 当strides大于1时需要：高宽/2,深度加深
    # 2. 对于18和34-layer: 第一层不需要虚线结构
    # 3. 对于50,101和152-layer: 第一层需要虚线结构：调整特征矩阵的深度，高宽不变.
    #                                           ->kernel_size=1
    skipLayer = None
    out_channel = unit1_channel * block.Expansion  # conv3_channel
    if (strides != 1) or (in_channel != out_channel):
        skipLayer = Sequential([
            Conv2D(filters=out_channel, kernel_size=1, strides=strides,
                   use_bias=False, name="conv1"),
            BatchNormalization(momentum=0.9, epsilon=1.001e-5, name="BatchNorm")
        ], name="shortcut")  # 跳过层即捷径层

    layersList = []
    # 首先针对第一个单元进行处理
    layersList.append(block(filters=unit1_channel, strides=strides,
                            down_sample=skipLayer,
                            name="unit_1"))
    # 然后针对其他单元进行处理
    for index in range(1, block_num):  # 3 -> 1, 2
        layersList.append(block(filters=unit1_channel, strides=1,
                                down_sample=None,
                                name="unit_" + str(index + 1)))

    return Sequential(layersList, name=name)

因为`Conv1`中刚刚对网络输入进行了卷积和最大池化，还没有进行残差学习，此时直接下采样会损失大量信息；而后3个`ConvN_x`直接进行下采样时，前面的网络已经进行过残差学习了，所以可以直接进行下采样。

#### 定义ResNet网络结构

In [10]:
def resnet(block, block_num_list, height=224, width=224, num_classes=1000, include_top=True):
    """
    :param block: 可选择 Residual_Basic 或 Residual_Bottleneck
    :param block_num_list: 残差结构数量 输入为列表
    :param height: 输入高度像素
    :param width: 输入宽度像素
    :param num_classes:  标签的类别数量
    :param include_top: 
    :return: 
    """
    input = Input(shape=[height, width, 3], dtype="float32")
    # ---------------------
    Z = Conv2D(filters=64, kernel_size=7, strides=2,
               padding="SAME", use_bias=False, name="conv1")(input)
    Z = BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv1/BatchNorm")(Z)
    Z = ReLU()(Z)

    Z = MaxPool2D(pool_size=3, strides=2, padding="SAME")(Z)
    # ---------------------
    # 每调用一次make_layer()就生成对应`convN_x`的一系列残差结构
    # Z.shape对应上一层输出特征矩阵的shape对应[batch, height, weight, channel]
    # Z.shape[-1]代表 channel 深度
    Z = make_conv_x(block=block, block_num=block_num_list[0],
                    in_channel=Z.shape[-1], unit1_channel=64, name="block1")(Z)
    Z = make_conv_x(block=block, block_num=block_num_list[1],
                    in_channel=Z.shape[-1], unit1_channel=128, name="block2", strides=2)(Z)
    Z = make_conv_x(block, block_num_list[2], Z.shape[-1], 256, "block3", 2)(Z)
    Z = make_conv_x(block, block_num_list[3], Z.shape[-1], 512, "block4", 2)(Z)
    # ---------------------
    if include_top:  # 不使用迁移学习
        Z = GlobalAvgPool2D()(Z)  # 全局平局池化:结合了pool和flatten的功能
        Z = Dense(units=num_classes, name="logits")(Z)
        predict = Softmax()(Z)
    else:  # 使用迁移学习 可以在后面自定义所需要的层
        predict = Z

    model = Model(inputs=input, outputs=predict)
    return model


#### 定义不同的ResNet架构

In [11]:
# 定义ResNet-18
def resnet_18(height=224, width=224, num_classes=1000, include_top=True):
    model = resnet(block=Residual_Basic, 
                   block_num_list=[2, 2, 2, 2], 
                   height=height, width=width,
                   num_classes=num_classes, 
                   include_top=include_top)
    return model

In [12]:
# 定义ResNet-34
def resnet_34(height=224, width=224, num_classes=1000, include_top=True):
    model = resnet(Residual_Basic, [3, 4, 6, 3], height, width, num_classes, include_top)
    return model

In [13]:
# 定义ResNet-50
def resnet_50(height=224, width=224, num_classes=1000, include_top=True):
    model = resnet(Residual_Bottleneck, [3, 4, 6, 3], height, width, num_classes, include_top)
    return model

In [14]:
# 定义ResNet-101
def resnet_101(height=224, width=224, num_classes=1000, include_top=True):
    model = resnet(Residual_Basic, [3, 4, 23, 3], height, width, num_classes, include_top)
    return model

In [15]:
# 定义ResNet-152
def resnet_152(height=224, width=224, num_classes=1000, include_top=True):
    model = resnet(Residual_Basic, [3, 8, 36, 3], height, width, num_classes, include_top)
    return model

- 查看`ResNet-34`模型结构

In [16]:
model = resnet_34(num_classes=10)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 64)      9408      
_________________________________________________________________
conv1/BatchNorm (BatchNormal (None, 112, 112, 64)      256       
_________________________________________________________________
re_lu (ReLU)                 (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0         
_________________________________________________________________
block1 (Sequential)          (None, 56, 56, 64)        222720    
_________________________________________________________________
block2 (Sequential)          (None, 28, 28, 128)       111872

#### 使用迁移学习训练ResNet-50

1. 加载,预处理数据集

In [17]:
import tensorflow_datasets as tfds
dataset, info = tfds.load("Cifar10", as_supervised=True, with_info=True)

class_names = info.features["label"].names
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [18]:
train_set_raw = tfds.load("Cifar10", as_supervised=True)['train']
test_set_raw, valid_set_raw = tfds.load(
    "Cifar10",
    split=["test[:60%]", "test[60%:]"],
    as_supervised=True)

如果使用`迁移学习`,需要在图像预处理部分减去`ImageNet`所有图像的均值,即**\[123.68, 116.78, 103.94\]** 如果使用别人的预训练模型参数,就必须和別人使用相同的预处理方法!

In [19]:
from keras.utils import np_utils

_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94

num_classes = 10


# 预处理
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224]) - [_R_MEAN, _G_MEAN, _B_MEAN]
    label = tf.cast(label, dtype=tf.int32)
    label = tf.squeeze(label)   # tf.squeeze():用于从张量形状中移除大小为1的维度
    label = tf.one_hot(label, depth=num_classes)
    return resized_image, label

batch_size = 64
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [20]:
root_logdir = os.path.join(os.curdir, "./Logs/my_ResNet50_logs")
root_logdir

def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_dir = get_run_logdir()
run_dir

tensorboard_cb = keras.callbacks.TensorBoard(run_dir)

2. 使用迁移学习

    载入权重后在原网络基础上再添加两层全连接层，仅训练最后两层全连接层

In [21]:
feature = resnet_50(num_classes=num_classes, include_top=False)

In [22]:
# 加载预训练模型的权重
pre_weight_path = './PTmodel/tf_resnet50_weights/pretrain_weights.ckpt'
feature.load_weights(pre_weight_path)
feature.trainable = False      # 冻结预训练模型的权重参数
feature.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 64)      9408      
_________________________________________________________________
conv1/BatchNorm (BatchNormal (None, 112, 112, 64)      256       
_________________________________________________________________
re_lu_17 (ReLU)              (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
block1 (Sequential)          (None, 56, 56, 256)       218624    
_________________________________________________________________
block2 (Sequential)          (None, 28, 28, 512)       1226

In [23]:
# 模型最后面添加全连接层
model = Sequential([
    feature,
    GlobalAvgPool2D(),
    Dropout(rate=0.5),
    Dense(1024, activation=keras.activations.relu),
    Dropout(rate=0.5),
    Dense(num_classes),
    Softmax()  
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Functional)         (None, 7, 7, 2048)        23561152  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
_________________________________________________________________
softmax_1 (Softmax)          (None, 10)                0

3. 训练,评估模型

In [ ]:
# 编译模型
optimizer = keras.optimizers.Adam(learning_rate=0.0002)
model.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
# 训练模型
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='./models/my_ResNet50.h5',
                                                save_best_only=True,
                                                ave_weights_only=True,
                                                monitor='val_loss'),
             tensorboard_cb]

history = model.fit(train_set,
                    steps_per_epoch=int(train_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.4 * test_size / batch_size),
                    epochs=5,
                    callbacks=callbacks)

Epoch 1/5
  3/781 [..............................] - ETA: 23:47 - loss: 2.3710 - accuracy: 0.1667

In [ ]:
model = keras.models.load_model("./my_VGG16.h5")
model.evaluate(test_set)

In [ ]:
history_dict = history.history
train_loss = history_dict["loss"]
train_accuracy = history_dict["accuracy"]
val_loss = history_dict["val_loss"]
val_accuracy = history_dict["val_accuracy"]

In [ ]:
epochs = 100
# figure 1
plt.figure()
plt.plot(range(epochs), train_loss, label='train_loss')
plt.plot(range(epochs), val_loss, label='val_loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')

# figure 2
plt.figure()
plt.plot(range(epochs), train_accuracy, label='train_accuracy')
plt.plot(range(epochs), val_accuracy, label='val_accuracy')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./Logs/my_VGG16_logs --port=6061